# Importing & Installing Libraries/Dependencies

#### Installing

In [ ]:
!pip install censusdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.6 MB 2.1 MB/s 
  Created wheel for censusdata: filename=CensusData-1.15.post1-py3-none-any.whl size=28205763 sha256=683ee45497d9030d200d993577ee5d47bf676f3a32c4c6b514e66656def1cc74
  Stored in directory: /root/.cache/pip/wheels/2f/72/16/5c8e185711751db7fa81da8c6329c8b3d82c0582a51a0f1e85
Successfully built censusdata


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Importing

In [ ]:
import pandas as pd
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

import requests

In [ ]:
#reading the API Key

path = '/content/drive/MyDrive/298DataFiles/DATA/ML2/CensusAPIkey.txt'
with open(path) as key:
    api_key=key.read().strip()

# Location details

In [ ]:
#censusgeo((('state', '06'),))
censusdata.geographies(censusdata.censusgeo([('state', '06'), ('county', '*')]), 'acs5', 2015)

{'Merced County, California': censusgeo((('state', '06'), ('county', '047'))),
 'Mariposa County, California': censusgeo((('state', '06'), ('county', '043'))),
 'Yuba County, California': censusgeo((('state', '06'), ('county', '115'))),
 'Contra Costa County, California': censusgeo((('state', '06'), ('county', '013'))),
 'Lassen County, California': censusgeo((('state', '06'), ('county', '035'))),
 'Stanislaus County, California': censusgeo((('state', '06'), ('county', '099'))),
 'Sonoma County, California': censusgeo((('state', '06'), ('county', '097'))),
 'Tehama County, California': censusgeo((('state', '06'), ('county', '103'))),
 'Mono County, California': censusgeo((('state', '06'), ('county', '051'))),
 'San Benito County, California': censusgeo((('state', '06'), ('county', '069'))),
 'Sacramento County, California': censusgeo((('state', '06'), ('county', '067'))),
 'El Dorado County, California': censusgeo((('state', '06'), ('county', '017'))),
 'Monterey County, California': c

In [ ]:
#Counties required = San Diego - 073,Los Angeles - 037,Del Norte - 015,Mendocino - 045,
#Sonoma- 097,Santa Barbara - 083,Monterey-053,San Mateo-081,Riverside -065,San Francisco -075,Ventura -111,San Luis Obispo
countyCode = ['073','037','015','045','097','083','053','081','065','075','111','079']

# 65+ population %

In [ ]:
#extracting for all counties for all years
years = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

#Counties required = San Diego - 073,Los Angeles - 037,Del Norte - 015,Mendocino - 045,
#Sonoma- 097,Santa Barbara - 083,Monterey-053,San Mateo-081,Riverside -065,San Francisco -075,Ventura -111,San Luis Obispo
countyCode = ['073','037','015','045','097','083','053','081','065','075','111','079']

PopChangeFactors = pd.DataFrame()

for i in years:
  for j in countyCode:
    PopChangeFactors_temp_df = censusdata.download('acs5', i, censusdata.censusgeo((('state', '06'), ('county', j))),
                                    ['B01001_001E', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E',
                                      'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E',
                                      'B01001_047E', 'B01001_048E', 'B01001_049E','B19013_001E','B01002_001E','C17002_002E','C17002_003E','C17002_004E','C17002_001E'])
    PopChangeFactors_temp_df['percent_65plus'] = (PopChangeFactors_temp_df.B01001_020E + PopChangeFactors_temp_df.B01001_021E + PopChangeFactors_temp_df.B01001_022E
                                    + PopChangeFactors_temp_df.B01001_023E + PopChangeFactors_temp_df.B01001_024E + PopChangeFactors_temp_df.B01001_025E
                                    + PopChangeFactors_temp_df.B01001_044E + PopChangeFactors_temp_df.B01001_045E + PopChangeFactors_temp_df.B01001_046E
                                    + PopChangeFactors_temp_df.B01001_047E + PopChangeFactors_temp_df.B01001_048E
                                    + PopChangeFactors_temp_df.B01001_049E) / PopChangeFactors_temp_df.B01001_001E * 100
    
    PopChangeFactors_temp_df['median_age'] = PopChangeFactors_temp_df['B01002_001E']
    PopChangeFactors_temp_df['median_HH_income'] = PopChangeFactors_temp_df['B19013_001E']
    PopChangeFactors_temp_df['percent_below_poverty'] = (PopChangeFactors_temp_df['C17002_002E'] + PopChangeFactors_temp_df['C17002_003E'] + PopChangeFactors_temp_df['C17002_004E']) / PopChangeFactors_temp_df['C17002_001E'] * 100
    PopChangeFactors_temp_df = PopChangeFactors_temp_df[['B01001_001E', 'percent_65plus','median_HH_income','median_age','percent_below_poverty']]
    PopChangeFactors_temp_df = PopChangeFactors_temp_df.rename(columns={'B01001_001E': 'population_size'})

    
    PopChangeFactors_temp_df['Year'] = i
    PopChangeFactors_temp_df['countyCode'] = j
    PopChangeFactors = PopChangeFactors.append(PopChangeFactors_temp_df)
  

print(PopChangeFactors.shape)
PopChangeFactors.head(3)

(132, 7)


,population_size,percent_65plus,median_HH_income,median_age,percent_below_poverty,Year,countyCode
"San Diego County, California: Summary level: 050, state:06> county:073",2987543,11.19,62901,34.5,15.75,2009,073
"Los Angeles County, California: Summary level: 050, state:06> county:037",9785295,10.30,54828,33.8,21.20,2009,037
"Del Norte County, California: Summary level: 050, state:06> county:015",28729,13.29,38408,37.4,25.84,2009,015


In [ ]:
# #Saving the Data
# path = '/content/drive/MyDrive/298DataFiles/DATA/ML2/PopChangeFactors.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   PopChangeFactors.to_csv(f)

In [ ]:
#aggregated family income
Employment_temp_df = censusdata.download('acs5', 2010, censusdata.censusgeo((('state', '06'), ('county', '073'))),
                                    ['B19130_001E'])

In [ ]:
# Search for ACS 2011-2015 5-year estimate variables where the specific variable label includes the text 'unemploy'.
censusdata.search('acs5', 2015, 'label', 'migra')

[('B25004_007E', 'VACANCY STATUS', 'Estimate!!Total!!For migrant workers')]

# Migration Data

In [ ]:
#extracting for all counties for 2013 & 2014
years = [2014,2013]

dsource='pep'
dname='cty'
cols='CTYNAME,DOM,NIM,DEATHS,BIRTHS,DATE_'
state='06'
county='073,037,015,045,097,083,053,081,065,075,111,079'

Migration_2013_14 = pd.DataFrame()


In [ ]:
for i in years:
  year = str(i)
  base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
  data_url = f'{base_url}?get={cols}&for=county:{county}&in=state:{state}&key={api_key}'

  response=requests.get(data_url)
  Migration_2013_14_temp=response.json()
  Migration_2013_14_temp_df=pd.DataFrame(Migration_2013_14_temp[1:], columns=Migration_2013_14_temp[0]).\
      rename(columns={ "DOM": "Net Domestic Mig", "NIM":"Net Foreign Mig"})
  Migration_2013_14_temp_df['fips']=Migration_2013_14_temp_df.state+Migration_2013_14_temp_df.county
  Migration_2013_14_temp_df.set_index('fips',inplace=True)
  Migration_2013_14_temp_df.drop(columns=['state'],inplace=True)
  Migration_2013_14_temp_df['Year'] = year
  Migration_2013_14 = Migration_2013_14.append(Migration_2013_14_temp_df)

print(Migration_2013_14.shape)
Migration_2013_14

(156, 8)


,CTYNAME,Net Domestic Mig,Net Foreign Mig,DEATHS,BIRTHS,DATE_,county,Year
fips,,,,,,,,
06015,Del Norte County,None,None,None,None,1,015,2014
06015,Del Norte County,None,None,None,None,2,015,2014
06015,Del Norte County,-50,3,93,107,3,015,2014
06015,Del Norte County,-146,15,295,335,4,015,2014
06015,Del Norte County,-317,14,284,324,5,015,2014
...,...,...,...,...,...,...,...,...
06111,Ventura County,None,None,None,None,2,111,2013
06111,Ventura County,109,363,1266,2804,3,111,2013
06111,Ventura County,-1540,1068,5030,11008,4,111,2013


In [ ]:
Migration_2013_14[(Migration_2013_14['DATE_']=='5') & (Migration_2013_14['county']=='073')]

,CTYNAME,Net Domestic Mig,Net Foreign Mig,DEATHS,BIRTHS,DATE_,county,Year
fips,,,,,,,,
06073,San Diego County,769,17028,19887,43840,5,073,2014
06073,San Diego County,409,14497,20536,43504,5,073,2013


In [ ]:
year

'2013'

In [ ]:
#extracting for all counties for 2013 & 2014
years = [2015,2016,2017,2018,2019]

dsource='pep'
dname='components'
cols='GEONAME,DOMESTICMIG,INTERNATIONALMIG,DEATHS,BIRTHS'
state='06'
county='073,037,015,045,097,083,053,081,065,075,111,079'



In [ ]:
migration_df_2015_19 = pd.DataFrame()
for i in years:
  year = str(i)
  base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
  data_url = f'{base_url}?get={cols}&for=county:{county}&in=state:{state}&key={api_key}'
  response=requests.get(data_url)
  try:
    data_15_19=response.json()
    migration_df=pd.DataFrame(data_15_19[1:], columns=data_15_19[0]).\
    rename(columns={ "DOMESTICMIG": "Net Domestic Mig", "INTERNATIONALMIG":"Net Foreign Mig"})
    migration_df['fips']=migration_df.state+migration_df.county
    migration_df.set_index('fips',inplace=True)
    migration_df.drop(columns=['state'],inplace=True)
    migration_df['Year'] = i
    migration_df_2015_19 = migration_df_2015_19.append(migration_df)  
  except ValueError:
    pass

migration_df_2015_19

,GEONAME,Net Domestic Mig,Net Foreign Mig,DEATHS,BIRTHS,county,Year
fips,,,,,,,
06075,"San Francisco County, California, Pacific, Wes...",1720,8476,6079,9059,075,2015
06079,"San Luis Obispo County, California, Pacific, W...",1205,275,2249,2734,079,2015
06081,"San Mateo County, California, Pacific, West, U...",-1623,5215,4976,9034,081,2015
06083,"Santa Barbara County, California, Pacific, Wes...",-1237,1715,3045,5883,083,2015
06097,"Sonoma County, California, Pacific, West, Unit...",404,717,4174,5116,097,2015
06015,"Del Norte County, California, Pacific, West, U...",-26,22,248,308,015,2015
06037,"Los Angeles County, California, Pacific, West,...",-61120,56684,64212,130070,037,2015
06045,"Mendocino County, California, Pacific, West, U...",-287,99,839,1025,045,2015
06053,"Monterey County, California, Pacific, West, Un...",-2880,1391,2484,6618,053,2015


In [ ]:
migration_df_2015_19[ migration_df_2015_19['county']=='073']

,GEONAME,Net Domestic Mig,Net Foreign Mig,DEATHS,BIRTHS,county,Year
fips,,,,,,,
06073,"San Diego County, California, Pacific, West, U...",-9372,18468,21580,44561,073,2015
06073,"San Diego County, California",-8300,14447,22552,44652,073,2016
06073,"San Diego County, California",-15784,15445,22715,43652,073,2017
06073,"San Diego County, California",-8394,18099,20620,44442,073,2018


In [ ]:
#Saving the Data
path = '/content/drive/MyDrive/298DataFiles/DATA/ML2/PopChangeFactors_Migration.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  MigrationData.to_csv(f)